## 1.get Chessboard corners

In [1]:
from src.get_corners import getChessboardCorners, normalize_points

In [2]:
# Chessboard의 image 좌표, world 좌표 load
chessboard_correspondences = getChessboardCorners(images=None, visualize = True)

Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042606.jpg
Chessboard Detected 
Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042608.jpg
Chessboard Detected 
Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042610.jpg
Chessboard Detected 
Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042612.jpg
Chessboard Detected 
Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042614.jpg
Chessboard Detected 
Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042616.jpg
Chessboard Detected 
Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042619.jpg
Chessboard Detected 
Processing Image :  /Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/IMG_20170209_042621.jpg
Chessboard Detected 
Processing Image :  /Use

## 2. chessboard 좌표 normalize

In [3]:
# Normalizing the chessboard points for better results of homography
chessboard_correspondences_normalized = normalize_points(chessboard_correspondences)

## 3.homography 연산

In [4]:
from src.calc_homogeneous import compute_view_based_homography, h_refined

In [5]:
# obtaining homography
homography = []
for correspondence in chessboard_correspondences_normalized:
    homography.append(compute_view_based_homography(correspondence))

Reprojection error :  1.0131843561129406
Reprojection error :  1.0082203742169988
Reprojection error :  1.0860946169237227
Reprojection error :  1.1750271524979705
Reprojection error :  0.9340955718388242
Reprojection error :  0.9969984852241014
Reprojection error :  1.0710749361594527
Reprojection error :  0.9558710463406693
Reprojection error :  0.9955951759652899
Reprojection error :  0.9434112270219359
Reprojection error :  1.1064929638716887
Reprojection error :  1.1760890142494036
Reprojection error :  1.0120425182721253


## 4. refine homography
- optimizing

In [10]:
# refining the obtsined homography
homography_refined = []
for i in range(len(homography)):
    h_opt = h_refined(homography[i], chessboard_correspondences_normalized[i])
    homography_refined.append(h_opt)

## 5. calibration matrix 연산

In [11]:
from src.calc_calibration import get_intrinsic_parameters, estimateExtrinsicParams

In [12]:
# obtaining the calibration matrix
K = get_intrinsic_parameters(homography_refined)
print("camera calibration matrix: ", K)

extrinsic_para = []
for i in range(len(homography_refined)):
    extrinsic = estimateExtrinsicParams(K, homography_refined[i])
    extrinsic_para.append(extrinsic)
    print("extrinsic params: ", extrinsic)

[1360.7609389701554, -0.6324837708339514, 2057.5956321718427, 2044.5774366042053, -0.6683900115445063, 764.0173982268695]
camera calibration matrix:  [[ 2.05759563e+03 -6.68390012e-01  7.64017398e+02]
 [ 0.00000000e+00  2.04457744e+03  1.36076094e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
extrinsic params:  [[ 0.034951    0.99870827  0.03187365 -2.77338574]
 [-0.98195026  0.02793704  0.18696121  0.31959841]
 [ 0.18588199 -0.03774798  0.98165827 17.28972538]]
extrinsic params:  [[ 3.41593226e-02  1.00090669e+00 -1.02806126e-02 -3.05890743e+00]
 [-9.44205141e-01  3.53021970e-02  3.27925192e-01  8.85336218e-01]
 [ 3.27581735e-01 -1.35960114e-03  9.46267138e-01  1.56420349e+01]]
extrinsic params:  [[ 0.04078828  0.99720876 -0.06079144 -2.90030839]
 [-0.89698641  0.06136165  0.43740759 -0.0881411 ]
 [ 0.44017236  0.03766137  0.89698554 14.71808679]]
extrinsic params:  [[-2.94921007e-03  9.97920211e-01 -7.64928395e-02 -2.75032729e+00]
 [-8.53188280e-01  3.38348029e-02  5.207133

## 6. distortion 연산
- 최종적으로 calibration 연산한 objecto point 추정

In [13]:
from src.calc_distortion import estimateReprojectionErrorDistortion

In [14]:
optpoints = []
for i in range(len(chessboard_correspondences)):
    image_points, object_points = chessboard_correspondences[i]
    points = estimateReprojectionErrorDistortion(K, extrinsic_para[i], image_points, object_points)
    optpoints.append(points)
optpoints = np.array(optpoints)

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_58671/486309817.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  optpoints = np.array(optpoints)


## 7. 결과 시각화, 저장

In [6]:
from src.visualization import visualize_pts

In [60]:
import glob

DATA_DIR = '/Users/ppangppang/Documents/dev/cv/data/Calibration_Imgs/'

images = [each for each in glob.glob(DATA_DIR + "*.jpg")]
images = sorted(images)
for i in range(len(optpoints)):
    image_points, object_points = chessboard_correspondences[i]
    visualize_pts(image_points, optpoints, images[i], i)